## transfer learning (전이 학습)
## transferlearning_001.ipynb

#### 최소한의 노력으로 최대의 효과를 얻고자 하는 것이 목표
#### pypi.org에서 수 많은 라이브러리가 있는 것처럼,
#### 분명히 다른 사람이 내가 하고자 하는 작업에 적합한 모델을 만드는데 시간을 보냈을 것이다.
#### 누군가가 딥러닝을 위한 모델을 이미 만들어서 존재 한다.

[Tensorflow hub](https://tfhub.dev/) : 기존 모델 구성을 위한 저장소

### 전의학습의 사용이유
#### 이미지 데이터를 구하기 어렵고, 구하는 시간도 오래걸린다.
#### 이미지를 구할 때 귀찮은 bias가 생실 수도 있다. (내가 좋은 결과를 얻고자 하는 이미지만 보려고 한다.)
#### 시간은 없고, 돈도 없고, 이미지는 적게 가지고 있는데 결과를 빨고 얻고 싶을때 

In [6]:
import zipfile


In [3]:
!wget -q https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip


In [7]:
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip", "r")
zip_ref.extractall()
zip_ref.close()

# 10_food_classes_10_precent 라는 폴더가 생성

In [9]:
import os

for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
    print(f"{dirpath} 폴더에는 {len(dirnames)}개의 폴더와 {len(filenames)}개의 파일이 존재한다.")

# 학습은 지난번보다 10%의 데이터로 하고, 테스트는 원래의 양과 같이 한다. 

10_food_classes_10_percent 폴더에는 2개의 폴더와 0개의 파일이 존재한다.
10_food_classes_10_percent/test 폴더에는 10개의 폴더와 0개의 파일이 존재한다.
10_food_classes_10_percent/test/ice_cream 폴더에는 0개의 폴더와 250개의 파일이 존재한다.
10_food_classes_10_percent/test/chicken_curry 폴더에는 0개의 폴더와 250개의 파일이 존재한다.
10_food_classes_10_percent/test/steak 폴더에는 0개의 폴더와 250개의 파일이 존재한다.
10_food_classes_10_percent/test/sushi 폴더에는 0개의 폴더와 250개의 파일이 존재한다.
10_food_classes_10_percent/test/chicken_wings 폴더에는 0개의 폴더와 250개의 파일이 존재한다.
10_food_classes_10_percent/test/grilled_salmon 폴더에는 0개의 폴더와 250개의 파일이 존재한다.
10_food_classes_10_percent/test/hamburger 폴더에는 0개의 폴더와 250개의 파일이 존재한다.
10_food_classes_10_percent/test/pizza 폴더에는 0개의 폴더와 250개의 파일이 존재한다.
10_food_classes_10_percent/test/ramen 폴더에는 0개의 폴더와 250개의 파일이 존재한다.
10_food_classes_10_percent/test/fried_rice 폴더에는 0개의 폴더와 250개의 파일이 존재한다.
10_food_classes_10_percent/train 폴더에는 10개의 폴더와 0개의 파일이 존재한다.
10_food_classes_10_percent/train/ice_cream 폴더에는 0개의 폴더와 75개의 파일이 존재한다.
10_food_classes_10_percent/train/chicken_curry

In [11]:
# 데이터가 준비가 된 상태
# 데이터를 분석할 준비하기

from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

# data normalization
train_datagen = ImageDataGenerator(rescale = 1/255.) # 값을 0 ~ 1 사이로 조정
test_datagen = ImageDataGenerator(rescale = 1/255.)

print("학습 이미지 : ")
train_data_10_percent = train_datagen.flow_from_directory(
    train_dir,
    target_size = IMAGE_SHAPE,
    batch_size = BATCH_SIZE,
    class_mode = "categorical"
)

print("테스트 이미지 : ")
test_data_10_percent = test_datagen.flow_from_directory(
    test_dir,
    target_size = IMAGE_SHAPE,
    batch_size = BATCH_SIZE,
    class_mode = "categorical"
)

학습 이미지 : 
Found 750 images belonging to 10 classes.
테스트 이미지 : 
Found 2500 images belonging to 10 classes.


### callback 설정 (모델이 학습하는 동안 실행하는 것)

#### 학습 중에 또는 학습 후에 수행할 모델에 기능을 더 추가해 주는 것.

#### TensorBoard 사용
#### 실험을 추적하고 여러 모델의 성능을 기록한 다음 tensorboard에서 시각적 방식으로 모델을 비교할 수 있다. 같은 데이터로 여러 모델의 결과를 비교하는데 유용하다.

#### 모델 체크포인트 (Model Checkpointing) : 상황에 따라 학습을 중지하고 다시 돌아와서 계속 진행할 수 있도록 모델을 저장

#### 조기 중지 (Early Stopping) : 임의의 시간동안 모델 학습을 진행시키다가 모델이 개선된 것으로 판단이 되면 학습을 자동으로 중단. 대용량 데이터셋이 있고 학습에 얼마나 오래 걸릴지 모를 때 유용한 개념


In [41]:
import datetime

def create_tensorboard_callback(dir_name, experiment_name):
    log_dir = dir_name + "/" + experiment_name + "/" +datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir
    )
    print(f"TensorBoard 로그 파일을 저장한 디렉토리 : {log_dir}")
    return tensorboard_callback

    

In [56]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [43]:
!pip install tensorflow_hub
!pip freeze > requirements.txt

In [44]:
import tensorflow_hub as hub
from tensorflow.keras import layers

In [45]:
# Resnet 50 V2
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

# EfficientNet0
efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

In [46]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


In [47]:
# 모델을 만드는 함수
# tensorflow hub url을 가지고 와서 keras sequential model을 생성
# model_url : tensorflow hub에 존재하는 model의 링크
# num_classese : 출력출에서 출력 뉴런의 갯수를 지정, 대상 클래스의 수와 같아야 한다.
# 아래 함수의 결과는 컴파일 되지 않은 keras sequential model
def create_model(model_url, num_classes = 10):
    feature_extractor_layer = hub.KerasLayer(
        model_url,
        trainable = False,  # 기본 패턴을 고정
        name = "feature_extraction_layer",
        input_shape = IMAGE_SHAPE + (3, )
    )

    model = tf.keras.Sequential([
        feature_extractor_layer,
        layers.Dense(
            num_classes, 
            activation = "softmax",
            name = "output_layer"
            )
    ])
    
    return model

In [53]:
def plot_loss_curves(history):
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]

    accuracy = history.history["accuracy"]
    val_accuracy = history.history["val_accuracy"]

    epochs = range(len(history.history["loss"]))

    plt.plot(epochs, loss, label="training loss")
    plt.plot(epochs, val_loss, label="validation loss")
    plt.title("Loss")
    plt.xlabel("Epochs")
    plt.legend()

    plt.figure()
    plt.plot(epochs, accuracy, label="training accuracy")
    plt.plot(epochs, val_accuracy, label="validation accuracy")
    plt.title("Accuracy")
    plt.xlabel("Epochs")
    plt.legend()

In [48]:
# 모델 생성
resnet_model = create_model(
    resnet_url,
    num_classes = train_data_10_percent.num_classes
)

# 컴파일
resnet_model.compile(
    loss = "categorical_crossentropy",
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)



In [49]:
resnet_history = resnet_model.fit(
    train_data_10_percent,
    epochs=5,
    steps_per_epoch=len(train_data_10_percent),
    validation_data = test_data_10_percent,
    validation_steps=len(test_data_10_percent),
    callbacks=[
        create_tensorboard_callback(
            dir_name="tensorflow_hub",
            experiment_name="resnet50V2"
        )
    ]
)

TensorBoard 로그 파일을 저장한 디렉토리 : tensorflow_hub/resnet50V2/20210619-105820
Epoch 1/5
24/24 [==============================] - 497s 20s/step - loss: 1.9176 - accuracy: 0.3627 - val_loss: 1.2077 - val_accuracy: 0.6108
Epoch 2/5
24/24 [==============================] - 440s 19s/step - loss: 0.8917 - accuracy: 0.7360 - val_loss: 0.8598 - val_accuracy: 0.7160
Epoch 3/5
24/24 [==============================] - 505s 22s/step - loss: 0.6001 - accuracy: 0.8493 - val_loss: 0.7456 - val_accuracy: 0.7564
Epoch 4/5
24/24 [==============================] - 405s 17s/step - loss: 0.4604 - accuracy: 0.8880 - val_loss: 0.6917 - val_accuracy: 0.7692
Epoch 5/5
24/24 [==============================] - 212s 9s/step - loss: 0.3749 - accuracy: 0.9107 - val_loss: 0.6717 - val_accuracy: 0.7752


In [54]:
plot_loss_curves(resnet_history)

In [50]:
# efficientnet 모델 생성
efficientnet_model = create_model(
    efficientnet_url,
    num_classes = train_data_10_percent.num_classes
)

# 컴파일
efficientnet_model.compile(
    loss = "categorical_crossentropy",
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)



In [51]:
efficientnet_history = efficientnet_model.fit(
    train_data_10_percent,
    epochs=5,
    steps_per_epoch=len(train_data_10_percent),
    validation_data = test_data_10_percent,
    validation_steps=len(test_data_10_percent),
    callbacks=[
        create_tensorboard_callback(
            dir_name="tensorflow_hub",
            experiment_name="efficientnet"
        )
    ]
)

TensorBoard 로그 파일을 저장한 디렉토리 : tensorflow_hub/efficientnet/20210619-113256
Epoch 1/5
24/24 [==============================] - 118s 5s/step - loss: 1.8608 - accuracy: 0.4213 - val_loss: 1.3148 - val_accuracy: 0.7244
Epoch 2/5
24/24 [==============================] - 104s 4s/step - loss: 1.0591 - accuracy: 0.7800 - val_loss: 0.8707 - val_accuracy: 0.8192
Epoch 3/5
24/24 [==============================] - 103s 4s/step - loss: 0.7509 - accuracy: 0.8493 - val_loss: 0.7002 - val_accuracy: 0.8384
Epoch 4/5
24/24 [==============================] - 103s 4s/step - loss: 0.5987 - accuracy: 0.8693 - val_loss: 0.6083 - val_accuracy: 0.8528
Epoch 5/5
24/24 [==============================] - 104s 4s/step - loss: 0.5074 - accuracy: 0.8840 - val_loss: 0.5562 - val_accuracy: 0.8632


In [52]:
efficientnet_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
feature_extraction_layer (Ke (None, 1280)              4049564   
_________________________________________________________________
output_layer (Dense)         (None, 10)                12810     
Total params: 4,062,374
Trainable params: 12,810
Non-trainable params: 4,049,564
_________________________________________________________________


In [57]:
plot_loss_curves(efficientnet_history)

In [ ]:
!python -m tensorboard.main dev upload --logdir ./tensorflow_hub/ --name ResNet50V2 --one_shot